In [1]:
# Use black formatter
%load_ext lab_black

import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Print with 3 decimals
np.set_printoptions(formatter={"float": lambda x: "{0:0.3f}".format(x)})

#### Ejecicio #1:    Normalización
Muchos algoritmos de Machine Learning necesitan datos de entrada centrados y normalizados. Una normalización habitual es el z-score, que implica restarle la media y dividir por el desvío a cada feature de mi dataset. 

Dado un dataset X de n muestras y m features, implementar un método en numpy para normalizar con z-score. Pueden utilizar np.mean() y np.std().

In [2]:
random_data = np.random.uniform(low=-100, high=100, size=(5, 5))
random_data

array([[-26.134, 47.944, 0.798, -89.664, 41.143],
       [87.919, -91.369, -92.790, -33.269, -33.506],
       [99.791, 42.826, 57.661, 87.456, -53.895],
       [-3.368, -19.668, -14.602, 97.356, -73.319],
       [-55.080, 55.053, -56.461, 14.666, -97.142]])

In [3]:
def normalize(data):
    """Normalize data using z-score formula"""
    return (data - data.mean(axis=0)) / data.std(axis=0)


normalize(random_data)

array([[-0.753, 0.732, 0.427, -1.475, 1.790],
       [1.083, -1.757, -1.400, -0.683, 0.208],
       [1.275, 0.641, 1.537, 1.014, -0.224],
       [-0.386, -0.476, 0.126, 1.153, -0.635],
       [-1.219, 0.859, -0.691, -0.009, -1.140]])

#### Ejecicio #2:    Remover filas y columnas con NaNs en un dataset
Dado un dataset, hacer una función que, utilizando numpy, filtre las columnas y las filas que tienen NaNs.

In [4]:
file_path = "/tf/notebooks/CEIA-inteligencia_artificial/clase_3/clase3v2.csv"


def load_dataset_from_file(file_path):
    with open(file_path, "r") as f:
        data = list(csv.reader(f, delimiter=";"))
    return np.array(data, dtype=float)


data = load_dataset_from_file(file_path)
data[:5, :]

array([[3.670, 2.864, nan, 2.949, nan, -9.365, 7.565],
       [13.505, 4.482, nan, 0.771, nan, -3.706, 32.867],
       [-5.737, -1.031, nan, 0.908, nan, 5.333, -20.922],
       [-0.019, 1.910, nan, 0.137, nan, 3.400, 1.433],
       [6.080, 1.528, nan, 0.746, nan, -11.487, 11.868]])

In [5]:
remove_nan_column = lambda data: data[:, ~np.isnan(data).any(axis=0)]
remove_nan_row = lambda data: data[~np.isnan(data).any(axis=1)]

print(f"Tamaño original: {data.shape}")
print(f"Tamaño sin NaN en columnas: {remove_nan_column(data).shape}")
print(f"Tamaño sin NaN en filas: {remove_nan_row(data).shape}")

Tamaño original: (100, 7)
Tamaño sin NaN en columnas: (100, 5)
Tamaño sin NaN en filas: (75, 7)


#### Ejecicio #3:    Reemplazar NaNs por la media de la columna
Dado un dataset, hacer una función que utilizando numpy reemplace los NaNs por la media de la columna.

In [6]:
def replace_nan_with_mean(data):
    """Replace NaN values with mean of the column"""
    mean = np.nanmean(data, axis=0)
    return np.nan_to_num(data, nan=mean)


replace_nan_with_mean(data)[:5, :]

array([[3.670, 2.864, 4.185, 2.949, -0.631, -9.365, 7.565],
       [13.505, 4.482, 4.185, 0.771, -0.631, -3.706, 32.867],
       [-5.737, -1.031, 4.185, 0.908, -0.631, 5.333, -20.922],
       [-0.019, 1.910, 4.185, 0.137, -0.631, 3.400, 1.433],
       [6.080, 1.528, 4.185, 0.746, -0.631, -11.487, 11.868]])

#### Ejecicio #4:    Dado un dataset X separarlo en 70 / 20 / 10
Como vimos en el ejercicio integrador, en problemas de Machine Learning es fundamental que separemos los datasets de n muestras, en 3 datasets de la siguiente manera:

* Training dataset: los datos que utilizaremos para entrenar nuestros modelos. Ej: 70% de las muestras.
* Validation dataset: los datos que usamos para calcular métricas y ajustar los hiperparámetros de nuestros modelos. Ej: 20% de las muestras.
* Testing dataset: una vez que entrenamos los modelos y encontramos los hiperparámetros óptimos de los mísmos, el testing dataset se lo utiliza para computar las métricas finales de nuestros modelos y analizar cómo se comporta respecto a la generalización. Ej: 10% de las muestras.

A partir de utilizar np.random.permutation, hacer un método que dado un dataset, devuelva los 3 datasets como nuevos numpy arrays.

In [7]:
def split_data(data, train_size=0.8, validation_size=None):
    """Split data into train, validation and test sets. Validation in optional"""

    if validation_size is None:
        if train_size > 1:
            raise ValueError("Train size must be less than 1")
    else:
        if train_size + validation_size > 1:
            raise ValueError("Train size + validation size must be less than 1")

    suffled_data = np.random.permutation(data)
    data_samples = data.shape[0]
    train_number = int(data_samples * train_size)

    if validation_size:
        validation_number = train_number + int(data_samples * validation_size)
        return (
            suffled_data[0:train_number],
            suffled_data[train_number:validation_number],
            suffled_data[validation_number:],
        )
    else:
        return suffled_data[0:train_number], suffled_data[train_number:]


train, validation, test = split_data(data, train_size=0.7, validation_size=0.2)
print(f"Tamaño del dataset train: {train.shape}")
print(f"Tamaño del dataset validation: {validation.shape}")
print(f"Tamaño del dataset test: {test.shape}")

Tamaño del dataset train: (70, 7)
Tamaño del dataset validation: (20, 7)
Tamaño del dataset test: (10, 7)


#### Ejercicio #5:   A partir del dataset de consigna, aplicar los conceptos de regresión lineal.
1. Cargar los datos con objeto de clase Data (implementada por ustedes) con un método que cumpla esa función al pasarle la ruta. Hacer un split de los datos en train/test (usar 80/20)
Tratar los nans con al menos dos de las técnicas vistas en clase. (pasarían a tener dos datasets para comparar en lo que sigue)
2. Utilizar PCA para quedarse con las 3 CP.  (de cada uno del punto 1, idealmente usen su implementación, pero pueden usar las librerías)
3. Crear una clase métrica base y una clase MSE que herede es ella. (esto viene de ejercicios anteriores)
4. Crear una clase modelo base y clase regresión lineal que herede de ella.  (esto viene de ejercicios anteirores)
5. Entrenar la regresión lineal sobre train. Calcular MSE sobre validation. (para todas las variantes que hayan hecho en 2) y comparar.

1.

In [8]:
class Data:
    """
    Class to load and prepare data for machine learning algorithms
    """

    def __init__(self, file):
        self.load_data(file)

    def load_data(self, file):
        self.original_data = load_dataset_from_file(file)
        self.data = self.original_data
        return self.data

    def fill_nans(self):
        self.data = replace_nan_with_mean(self.data)

    def remove_nans(self):
        self.data = remove_nan_column(self.data)

    def normalize(self):
        self.data = normalize(self.data)

    def prepare_data(self, nan_strategy="fill"):
        if nan_strategy == "fill":
            self.fill_nans()
        elif nan_strategy == "remove":
            self.remove_nans()
        self.normalize()

    def split_data(self, train_size=0.8, validation_size=None):
        return split_data(
            self.data, train_size=train_size, validation_size=validation_size
        )

    def restore_data(self):
        self.data = self.original_data

    def get_data(self):
        return self.data


data = Data(file_path)
strategies = ["fill", "remove"]
dataArray = []

for strategy in strategies:
    data.prepare_data(nan_strategy=strategy)
    train, test = data.split_data(train_size=0.8)
    dataArray.append(
        {
            "train": {
                "X": train[:, :-1],
                "y": train[:, -1],
            },
            "test": {
                "X": test[:, :-1],
                "y": test[:, -1],
            },
        }
    )
    data.restore_data()

2.

In [9]:
# Agrega "X_pca" de 3 componentes a los datos de train y test
for data in dataArray:
    for key in data:
        X_pca = PCA(n_components=3).fit_transform(data[key]["X"])
        data[key]["X_pca"] = X_pca

3.

In [10]:
class BaseMetric:
    """
    Abstract class for metrics
    """

    def __init__(self, name):
        self.name = name

    def __call__(self):
        raise NotImplementedError("__call__ method not implemented")


class MSE(BaseMetric):
    """
    Abstract class for metrics
    """

    def __init__(self):
        super().__init__("Mean Squared Error")

    def __call__(self, y_true, y_pred):
        return np.sum(y_true - y_pred) ** 2 / y_true.shape[0]

4.

In [11]:
class Model:
    """
    Abstract class for models
    """

    def __init__(self):
        self.model = None


class LinearRegression(Model):
    """
    Class for Linear Regression Model
    """

    def fit(self, X, y):
        try:
            self.model = np.linalg.inv(X.T @ X) @ X.T @ y
        except:
            self.model = np.linalg.pinv(X.T @ X) @ X.T @ y

    def predict(self, X):
        return X @ self.model

5.

In [38]:
linearRegression = LinearRegression()
mse = MSE()

for data in dataArray:
    linearRegression.fit(data["train"]["X_pca"], data["train"]["y"])
    y_pred = linearRegression.predict(data["test"]["X_pca"])
    data["test"]["MSE"] = mse(data["test"]["y"], y_pred)

print(f'MSE: {dataArray[0]["test"]["MSE"]:.3f} (NaN promediados)')
print(f'MSE: {dataArray[1]["test"]["MSE"]:.3f} (NaN eliminados)')
print(
    """
El valor de MSE es menor en el dataset en el que se eliminaron las
columnas con NaN. Esto puede deberse a que dichas columnas no aportaban
información importante y en el primer caso solo sirvieron para empeorar
la performance del modelo. También puede ser que el promedio (reemplazo
para los NaN), no sea un valor representativo para el conjunto de datos."""
)

MSE: 1.356 (NaN promediados)
MSE: 0.247 (NaN eliminados)

El valor de MSE es menor en el dataset en el que se eliminaron las
columnas con NaN. Esto puede deberse a que dichas columnas no aportaban
información importante y en el primer caso solo sirvieron para empeorar
la performance del modelo. También puede ser que el promedio (reemplazo
para los NaN), no sea un valor representativo para el conjunto de datos.
